In [1]:
#import dependencies
import gmaps
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

In [2]:
path = '../weatherpy/outputs/cities.csv'
cities_for_heatmap_df = pd.read_csv(path)
cities_for_heatmap_df.sample(10)

,Unnamed: 0,city_name,lat,long,max_temp,cloudiness,humidity,wind_speed,country_code,date_of_measurement
337,337,comodoro rivadavia,-45.87,-67.50,60.80,75,44,5.82,AR,1608823828
273,273,port augusta,-32.50,137.77,64.40,0,77,12.75,AU,1608823816
62,62,trat,12.50,102.50,75.76,32,77,5.48,TH,1608823789
251,251,dok kham tai,19.16,99.99,62.62,7,74,0.83,TH,1608823800
383,383,grand-santi,4.25,-54.38,82.72,100,76,4.88,GF,1608823832
525,525,sinop,41.77,34.87,35.42,0,76,7.54,TR,1608823771
257,257,ambon,-3.70,128.18,84.99,100,84,4.14,ID,1608823805
46,46,half moon bay,37.46,-122.43,46.40,1,86,4.70,US,1608823788
35,35,guerrero negro,27.98,-114.06,60.96,100,41,3.80,MX,1608823787
316,316,araouane,18.90,-3.53,83.21,100,14,7.25,ML,1608823826


In [3]:
cities_for_heatmap_df['humidity'].max()

100

In [4]:
locations = cities_for_heatmap_df[['lat', 'long']]
weights = cities_for_heatmap_df['humidity']
central_coordinates = (0, 0)
fig = gmaps.figure(center=central_coordinates, zoom_level=1.5)
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights, max_intensity=100, point_radius=6)
heatmap_layer.opacity = 0.5
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
ideal_weather_df = pd.DataFrame(cities_for_heatmap_df[(cities_for_heatmap_df['max_temp'] < 70) & (cities_for_heatmap_df['max_temp'] > 60) & (cities_for_heatmap_df['wind_speed'] < 10) & (cities_for_heatmap_df['cloudiness'] == 0)])
ideal_weather_df['hotel_name'] = " "
ideal_weather_df

,Unnamed: 0,city_name,lat,long,max_temp,cloudiness,humidity,wind_speed,country_code,date_of_measurement,hotel_name
15,15,bredasdorp,-34.53,20.04,69.80,0,56,8.05,ZA,1608823786,
102,102,tiznit,29.58,-9.50,66.16,0,27,6.51,MA,1608823791,
368,368,magway,20.15,94.92,67.59,0,62,3.40,MM,1608823831,
426,426,forbes,-33.38,148.02,64.99,0,52,7.23,AU,1608823836,
480,480,shadegan,30.65,48.66,60.80,0,72,9.17,IR,1608823841,
487,487,pathein,16.78,94.73,67.95,0,79,5.26,MM,1608823823,
493,493,lom sak,16.78,101.24,68.95,0,76,3.80,TH,1608823842,
548,548,bani walid,31.76,13.99,62.22,0,50,5.14,LY,1608823847,
551,551,turbat,26.00,63.04,63.84,0,30,4.43,PK,1608823847,


In [6]:
hotel_names = []
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
key=g_key
target_radius = 5000
target_keyword = 'hotel'
target_type = 'lodging'
for index, row in ideal_weather_df.iterrows():
    try:
        target_location = f'{row[1]},{row[2]}'
        final_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={target_location}&radius={target_radius}&keyword={target_keyword}&type={target_type}&key={api_key}'
        data = requests.get(final_url).json()
        hotel_names.append(data['results'][0]['name'])
    except IndexError:
        print(f'No hotel within 5000 m of {row[0]}, must drop row')
        hotel_names.append('n/a')
ideal_weather_df['hotel_name'] = hotel_names
ideal_weather_df

NameError: name 'api_key' is not defined

In [ ]:
hotels_df = ideal_weather_df.drop(ideal_weather_df[ideal_weather_df.hotel_name == 'n/a'].index)
hotels_df

In [ ]:
locations = cities_for_heatmap_df[['lat', 'long']]
weights = cities_for_heatmap_df['humidity']
central_coordinates = (0, 0)
fig = gmaps.figure(center=central_coordinates, zoom_level=1.5)
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights, max_intensity=100, point_radius=2, dissipating=False)
heatmap_layer.opacity = 0.5
fig.add_layer(heatmap_layer)
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel_name}</dd>
<dt>City</dt><dd>{city_name}</dd>
<dt>Country</dt><dd>{country_code}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]
hotel_locations = hotels_df[["lat", "long"]]


marker_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

In [ ]:
hotel_names = []
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
key=g_key
target_radius = 5000
target_keyword = 'hotel'
target_type = 'lodging'
for index, row in ideal_weather_df.iterrows():
    try:
        target_location = f'{row[1]},{row[2]}'
        final_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={target_location}&radius={target_radius}&keyword={target_keyword}&type={target_type}&key={api_key}'
        data = requests.get(final_url).json()
        hotel_names.append(data['results'][0]['name'])
    except IndexError:
        print(f'No hotel within 5000 m of {row[0]}, will try 10000 m range')
        try:
            target_radius = 10000
            target_location = f'{row[1]},{row[2]}'
            final_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={target_location}&radius={target_radius}&keyword={target_keyword}&type={target_type}&key={api_key}'
            data = requests.get(final_url).json()
            hotel_names.append(f"{data['results'][0]['name']}* 10 km away")
            target_radius = 5000
        except IndexError:
            print(f'No hotel within 10000 m of {row[0]}, will try 20000 m range')
            try:
                target_radius = 20000
                target_location = f'{row[1]},{row[2]}'
                final_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={target_location}&radius={target_radius}&keyword={target_keyword}&type={target_type}&key={api_key}'
                data = requests.get(final_url).json()
                hotel_names.append(f"{data['results'][0]['name']}* 20 km away")
                target_radius = 5000
            except IndexError:
                print(f'no hotel found within 20000 m of {row[0]}, exiting row')
                continue

ideal_weather_df['hotel_name'] = hotel_names
ideal_weather_df

In [ ]:
#testing to see if error is because of lack of hotels w/in 5000 m; increasing to 10000 m, then 20000 m
target_coords = [[29.58,-9.50], [30.65,48.66]]
hotel_names = []
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
key=g_key
target_radius = 20000
target_keyword = 'hotel'
target_type = 'lodging'
for item in target_coords:
    try:
        target_location = f'{item[0]},{item[1]}'
        final_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={target_location}&radius={target_radius}&keyword={target_keyword}&type={target_type}&key={api_key}'
        data = requests.get(final_url).json()
        hotel_names.append(data['results'][0]['name'])
    except IndexError:
        print(f'Error with {row[0]}')
        hotel_names.append('n/a')
        continue
hotel_names